In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import time


%load_ext autoreload
%autoreload 2

In [2]:
from helpers_create_data import *
from implementations import *

In [3]:
#We import the data here
from helpers import load_csv_data
x_train, x_test, y_train, train_ids, test_ids = load_csv_data(".", sub_sample=False)

In [4]:
# we drop the rows that has a NaN percentage of {threshold} because we assume that they don't offer much information
X_tr_all, Y_tr_all = drop_rows_with_nan(x_train, y_train, threshold=0.4)

X_train_all, X_val_all, Y_train_all, Y_val_all = split_train_val(X_tr_all, Y_tr_all, 10, 9)

# we process the dataset by replacing the remaining NaNs by column with the mode of the feature column that have less than 10 unique values and by its 
# mean if the feature column has more than 10 unique values. Also we remove the columns that have extremely low variance as this column 
# doesn't offer any information and we might encouter numerical issues when standardizing.
X_tr_all, X_val_all, X_test_all = process_datasets(X_train_all, X_val_all, x_test, unique_values_thresh=10)

# we replace the the -1s in the y column by 0s
Y_train_all[Y_train_all == -1] = 0
Y_val_all[Y_val_all == -1] = 0

# We standardize the datasets in order to give us better numerical results
X_tr_all, mean_x_tr_all, std_x_tr_all = standardize(X_tr_all)
X_val_all,_,_ = standardize(X_val_all, mean_x_tr_all, std_x_tr_all)
X_test_all,_,_ = standardize(X_test_all, mean_x_tr_all, std_x_tr_all)

# We now balance the data to a slightly more balanced ratio of 0s and 1s
X_tr_all, Y_tr_all = undersampling_oversampling(X_tr_all, Y_train_all, ratio_majority=0.5, ratio_majority_to_minority=2)

# We add a column of ones (bias term) to the dataset
X_tr_all = np.c_[np.ones((X_tr_all.shape[0], 1)), X_tr_all]
X_val_all = np.c_[np.ones((X_val_all.shape[0], 1)), X_val_all]
X_test_all = np.c_[np.ones((X_test_all.shape[0], 1)), X_test_all]

In [10]:
#Gamma = 0.01 was giving us good results while converging in acceptable time
gamma = 0.01
max_iter = 10000

#Reshape y_train form (#points,1) to (#points,) in order to use the implemented logistic regression function
Y_tr_all = Y_tr_all.reshape(-1)
#Create a new w in order to match the number of sected feature and has shape (1 + #features, )
w_reg = np.zeros((X_tr_all.shape[1], 1)).reshape(-1)
#Train model (-> our train set) using logistic regression
w, loss = logistic_regression(Y_tr_all, X_tr_all, w_reg, max_iter, gamma)

In [11]:
#y_pred_test are the predicted labels for the validation set
y_pred_test = prediction(X_val_all, w)
Y_val = Y_val_all.reshape(-1)
print('Accuracy:', compute_accuracy(Y_val, y_pred_test))
print('F1: ', f1(y_pred_test, Y_val))

Accuracy: 82.33387358184766
F1:  0.4315514993481095


In [12]:
#Here we actually make the prediction for the test set
y_pred = prediction(X_test_all, w)
y_pred[y_pred == 0] = -1

In [9]:
from helpers import create_csv_submission
create_csv_submission(test_ids, y_pred, "Submission_25.10.2024_16_20")